In [39]:
import numpy as np

def find_equilibrium(G, q, mode='all one', tol=1e-5, max_iter=1000):
    n = len(q)  # Number of agents
    x = np.zeros(n)  # Initial actions
    
    if mode == 'all one':
        for iteration in range(max_iter):
            x_new = np.zeros(n)
            for i in range(n):
                sum_gx = np.sum(G[i, :] * x)
                x_new[i] = max(0, q[i] - sum_gx)
            
            # Check for convergence
            if np.linalg.norm(x_new - x) < tol:
                print(f"Converged in {iteration+1} iterations.")
                return x_new
            x = x_new

    elif mode in ['one two unaware', 'one two aware']:
        G2 = np.linalg.matrix_power(G, 2)  # Compute G squared
        for iteration in range(max_iter):
            x_new = np.zeros(n)
            # Update x1 using G + G^2
            sum_g1x = np.sum((G[0, :] + G2[0, :]) * x)
            x_new[0] = max(0, q[0] - sum_g1x)
            
            if mode == 'one two unaware':
                # x_11 for Agent 1 using original G
                x_11 = max(0, q[0] - np.sum(G[0, :] * x))  
                # Update x2 specifically using x_11 and x3
                x_new[1] = max(0, q[1] - G[1, 0] * x_11 - G[1, 2] * x[2])
            elif mode == 'one two aware':
                # Update x2 and x3 using x_12 (Agent 1's action from one two aware mode)
                x_new[1] = max(0, q[1] - G[1, 0] * x_new[0] - G[1, 2] * x[2])
                x_new[2] = max(0, q[2] - G[2, 0] * x_new[0] - G[2, 1] * x[1])
            
            # Update x3 normally (assuming similar to x2)
            if mode == 'one two unaware':
                sum_g3x = np.sum(G[2, :] * x)
                x_new[2] = max(0, q[2] - sum_g3x)
            
            # Check for convergence
            if np.linalg.norm(x_new - x) < tol:
                print(f"Converged in {iteration+1} iterations.")
                return x_new
            x = x_new

    print("Max iterations reached without convergence.")
    return x

# Example adjacency matrix and q vector
G = np.array([[0, 0.7, 0.2],
              [0.1, 0, 0.5],
              [0.5, 0.3, 0]])

q = np.array([1, 1, 1])

# Testing all modes
print("Equilibrium actions for 'all one':")
equilibrium_all_one = find_equilibrium(G, q, mode='all one')
print(equilibrium_all_one)

print("\nEquilibrium actions for 'one two unaware':")
equilibrium_one_two = find_equilibrium(G, q, mode='one two unaware')
print(equilibrium_one_two)

print("\nEquilibrium actions for 'one two aware':")
equilibrium_one_two_aware = find_equilibrium(G, q, mode='one two aware')
print(equilibrium_one_two_aware)

Equilibrium actions for 'all one':
Converged in 43 iterations.
[0.41812103 0.66202293 0.59233701]

Equilibrium actions for 'one two unaware':
Converged in 72 iterations.
[0.12265268 0.57180212 0.76712875]

Equilibrium actions for 'one two aware':
Converged in 12 iterations.
[0.10000033 0.60588281 0.76823497]
